In [1]:
from proc_v1 import p, target, X_cat, X_cat2, X_num, X_all, sc
from ml_v1 import config, xgb_adapter, lgb_adapter, cb_adapter, nn_adapter, lr_adapter, scheduler, mapk_prob, ss, skf
from sklearn.preprocessing import LabelEncoder
from functools import partial
import sgml
import numpy as np
import pandas as pd
le = LabelEncoder()
target_l = target + '_l'
df_train = p.fit_transform(['data/train.csv'])
df_org = p.transform(['data/Fertilizer Prediction.csv'])
df_test = p.transform(['data/test.csv'])
df_train = df_train.assign(
    **{target_l: lambda x: le.fit_transform(x[target])}
)
df_org = df_org.assign(
    **{target_l: lambda x: le.transform(x[target])}
)

2025-06-12 21:51:25.401413: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 21:51:25.559153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749732685.615271    3536 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749732685.630760    3536 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749732685.747302    3536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# 원본 데이터 포함의 유용성 검사

## XGB1_ORG

In [2]:
result = sc.cv_result(
    'xgb1_org', df_train, skf, {
        'model_params': {'n_estimators': 1500, 'max_depth': 6, 'enable_categorical': True, 'learning_rate': 0.08}, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof'
    }, config, xgb_adapter, use_gpu = True, rerun = False,
    desc = '원본 데이터를 포함한 XGB 모델'
)

In [3]:
np.mean(result['valid_scores']), result['valid_scores']

(0.34894755555555557,
 [0.34786111111111107,
  0.3493533333333334,
  0.3484511111111111,
  0.3486333333333334,
  0.35043888888888886])

In [4]:
pd.concat([i['valid_result'] for i in result['model_result']], axis = 1).T.groupby(level = 1).mean().T.min()

set
validation_0    1.754444
validation_1    1.907721
dtype: float64

## XGB1_NOORG

In [5]:
result = sc.cv_result(
    'xgb1_noorg', df_train, skf, {
        'model_params': {'n_estimators': 1500, 'max_depth': 6, 'enable_categorical': True, 'learning_rate': 0.08}, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': None},
        'validation_fraction': 'oof'
    }, config, xgb_adapter, use_gpu = True, rerun = True,
    desc = '원본 데이터를 포함하지 않은 XGB 모델'
)

In [6]:
np.mean(result['valid_scores']), result['valid_scores']

(0.3411617777777778,
 [0.3400055555555555,
  0.34139222222222226,
  0.3415411111111111,
  0.34183222222222226,
  0.3410377777777778])

In [7]:
pd.concat([i['valid_result'] for i in result['model_result']], axis = 1).T.groupby(level = 1).mean().T.min()

set
validation_0    1.741575
validation_1    1.914536
dtype: float64

In [11]:
results = sc.read_cvs(['xgb1_org', 'xgb1_noorg'])

- 원본데이터는 상당히 유용합니다.

## Pilot Submission

In [17]:
sc.train_cv('xgb1_org', df_train, config, use_gpu = True)

(XGBClassifier(base_score=None, booster=None, callbacks=[XGBFitProgressbar],
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device='cuda', early_stopping_rounds=None,
               enable_categorical=True, eval_metric=None,
               feature_types=['c', 'c', 'q', 'q', 'q', 'q', 'q', 'q'],
               feature_weights=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=0.08, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=1500,
               n_jobs=None, num_parallel_tree=None, ...),
 ColumnTransformer(transformers=[('cat', OrdinalEncoder(dtype='int'),
                                  ['Soil_Type', 'Crop_Type']),
                              

In [42]:
df_prd = sc.get_predictor_cv('xgb1_org', config)(df_test)
pd.DataFrame(
    le.inverse_transform(np.argsort(-df_prd.values, axis=1)[:, :3].reshape(-1)).reshape(-1, 3),
    index = df_test.index
).apply(lambda x: x.str.cat(sep=' '), axis=1).rename(target.replace('_', ' ')).to_csv('result/submission0.csv')

In [43]:
# !kaggle competitions submit -c playground-series-s5e6 -f result/submission0.csv -m "Pilot"

100%|██████████████████████████████████████| 6.80M/6.80M [00:02<00:00, 2.84MB/s]
Successfully submitted to Predicting Optimal Fertilizers

- LB: 0.35029

## XGB2_ORG

[Single XGBoost Baseline](https://www.kaggle.com/code/lizzylee1111111111/single-xgboost-baseline)

하나의 모델로 좋은 성능을 보여주고 있습니다.

**발견**: 컬럼 샘플링이 상당한 성능 개선을 가져온 요소입니다.

In [21]:
result = sc.cv_result(
    'xgb2_org', df_train, skf, {
        'model_params': {
            'n_estimators': 4000, 'max_depth': 12, 'enable_categorical': True, 'learning_rate': 0.03,
            'subsample': 0.86, 'gamma': 0.26, 'max_delta_step': 4, 'reg_alpha': 2.7, 'reg_lambda': 1.4,
            'early_stopping_rounds': 100, 'objective': 'multi:softprob', 'colsample_bytree': 0.467, 'random_state': 13, 'tree_method': 'hist'
        }, 'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof',
    }, config, xgb_adapter, use_gpu = True, rerun = False, result_proc = partial(sgml.predict_learning_result, df=df_test),
    desc = '[Single XGBoost Baseline](https://www.kaggle.com/code/lizzylee1111111111/single-xgboost-baseline)'
)

In [5]:
np.mean(result['valid_scores']), result['valid_scores']

(0.36566866666666664,
 [0.36506666666666665,
  0.36647111111111114,
  0.3645744444444444,
  0.3651477777777778,
  0.3670833333333333])

In [13]:
df_prd = pd.concat([pd.DataFrame(i, index = df_test.index) for i in result['model_result']], axis=1).T.groupby(level = 0).mean().T
pd.DataFrame(
    le.inverse_transform(np.argsort(-df_prd.values, axis=1)[:, :3].reshape(-1)).reshape(-1, 3),
    index = df_test.index
).apply(lambda x: x.str.cat(sep=' '), axis=1).rename(target.replace('_', ' ')).to_csv('result/submission1.csv')

In [14]:
# !kaggle competitions submit -c playground-series-s5e6 -f result/submission1.csv -m "Pilot2"

100%|██████████████████████████████████████| 6.79M/6.79M [00:02<00:00, 2.77MB/s]
Successfully submitted to Predicting Optimal Fertilizers

- LB 0.36754

## XGB3_ORG

In [40]:
result = sc.cv_result(
    'xgb3_org', df_train, skf, {
        'model_params': {
            'n_estimators': 4500, 'max_depth': 6, 'enable_categorical': True, 'learning_rate': 0.07, 'colsample_bytree': 0.25
        }, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof'
    }, config, xgb_adapter, use_gpu = True, rerun = False,
    desc = '원본 데이터를 포함한 XGB 모델 + column sampling'
)

In [41]:
np.mean(result['valid_scores']), result['valid_scores']

(0.36093311111111115,
 [0.36016888888888887,
  0.3624611111111111,
  0.3607688888888889,
  0.36018222222222224,
  0.36108444444444443])

## XGB4_ORG

In [26]:
result = sc.cv_result(
    'xgb4_org', df_train, skf, {
        'model_params': {
            'n_estimators': 1850, 'max_depth': 12, 'enable_categorical': True, 'learning_rate': 0.03,
            'subsample': 0.86, 'gamma': 0.26, 'max_delta_step': 4, 'reg_alpha': 2.7, 'reg_lambda': 1.4,
            'early_stopping_rounds': 100, 'objective': 'multi:softprob', 'colsample_bytree': 0.467, 'random_state': 13, 'tree_method': 'hist'
        }, 'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof',
    }, config, xgb_adapter, use_gpu = True, rerun = False, result_proc = partial(sgml.predict_learning_result, df=df_test),
    desc = 'XGB1_ORG + fixed estimators'
)

In [28]:
np.mean(result['valid_scores']), result['valid_scores']

(0.36566866666666664,
 [0.36506666666666665,
  0.36647111111111114,
  0.3645744444444444,
  0.3651477777777778,
  0.3670833333333333])

## LGB1_ORG

In [55]:
result = sc.cv_result(
    'lgb1_org', df_train, skf, {
        'model_params': {
            'n_estimators': 1350, 'num_leaves': 255, 'learning_rate': 0.03, 'colsample_bytree': 0.4, 'min_child_samples': 15,
            'subsample': 0.75, 'subsample_freq': 1
        }, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof'
    }, {**config, 'return_train_scores': False}, lgb_adapter, rerun = False
)

In [56]:
np.mean(result['valid_scores']), result['valid_scores']

(0.3634862222222222,
 [0.36243000000000003,
  0.36451555555555554,
  0.3631188888888889,
  0.36284333333333335,
  0.36452333333333337])

## LGB2_ORG

- 컬럼 샘플링 비율에 상당히 민감하게 동작합니다. 0.4 

In [32]:
result = sc.cv_result(
    'lgb2_org', df_train, skf, {
        'model_params': {
            'n_estimators': 2000, 'num_leaves': 127, 'learning_rate': 0.03, 'colsample_bytree': 0.4, 'min_child_samples': 15,
            'subsample': 0.75, 'subsample_freq': 1
        }, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof'
    }, {**config, 'return_train_scores': False}, lgb_adapter, rerun = False
)

In [33]:
np.mean(result['valid_scores']), result['valid_scores']

(0.36229555555555554,
 [0.3608033333333333,
  0.3638855555555556,
  0.3619833333333333,
  0.36137777777777774,
  0.3634277777777778])

## LGB3_ORG

In [42]:
result = sc.cv_result(
    'lgb3_org', df_train, skf, {
        'model_params': {
            'n_estimators': 1200, 'num_leaves': 255, 'learning_rate': 0.03, 'colsample_bytree': 0.5, 'min_child_samples': 512
        }, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof'
    }, {**config, 'return_train_scores': False}, lgb_adapter, rerun = False,
    desc = '원본 데이터를 포함한 LGB 모델 + column sampling'
)

In [43]:
np.mean(result['valid_scores']), result['valid_scores']

(0.3547617777777778,
 [0.35392444444444443,
  0.3551622222222222,
  0.35506333333333334,
  0.3536533333333334,
  0.35600555555555563])

## LGB4_ORG

L1, L2 규제를 가해봅니다.

In [8]:
result = sc.cv_result(
    'lgb4_org', df_train, skf, {
        'model_params': {
            'n_estimators': 1350, 'num_leaves': 255, 'learning_rate': 0.03, 'colsample_bytree': 0.4, 'min_child_samples': 15,
            'subsample': 0.75, 'subsample_freq': 1, 'reg_alpha': 1, 'reg_lambda': 1
        }, 
        'X_num': X_num, 'X_cat': X_cat, 
        'train_data_proc_param': {'org': df_org},
        #'validation_fraction': 'oof'
    }, {**config, 'return_train_scores': False}, lgb_adapter, rerun = True
)

In [9]:
np.mean(result['valid_scores']), result['valid_scores']

(0.36375244444444443,
 [0.36299444444444445,
  0.36447000000000007,
  0.3633066666666667,
  0.36299111111111104,
  0.36500000000000005])

## CB1_ORG 

- CatBoost는 gpu를 활용하기엔 컬럼 샘플링을 사용할 수없는 여러 제약 사항이 있어 사용을 하지 않습니다.

## XGB2_ORG_CAT2

- Crop_Type과 Soil_Type을 합친 속성을 써봅니다.

In [33]:
result = sc.cv_result(
    'xgb2_org_cat2', df_train, skf, {
        'model_params': {
            'n_estimators': 4000, 'max_depth': 11, 'enable_categorical': True, 'learning_rate': 0.03,
            'subsample': 0.86, 'gamma': 0.26, 'max_delta_step': 4, 'reg_alpha': 1, 'reg_lambda': 0.1,
            'early_stopping_rounds': 100, 'objective': 'multi:softprob', 'colsample_bytree': 0.35, 'random_state': 13, 'tree_method': 'hist'
        }, 'X_num': X_num, 'X_cat': X_cat + X_cat2, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof',
    }, config, xgb_adapter, use_gpu = True, rerun = True, result_proc = partial(sgml.predict_learning_result, df=df_test),
    desc = '[Single XGBoost Baseline](https://www.kaggle.com/code/lizzylee1111111111/single-xgboost-baseline) + CAT2'
)

In [34]:
np.mean(result['valid_scores']), result['valid_scores']

(0.36039177777777776,
 [0.3594677777777778,
  0.36145999999999995,
  0.3601111111111111,
  0.35984,
  0.36108])

## LGB2_ORG_CAT2

In [71]:
result = sc.cv_result(
    'lgb2_org_cat2', df_train, skf, {
        'model_params': {
            'n_estimators': 2000, 'num_leaves': 127, 'learning_rate': 0.03, 'colsample_bytree': 0.35, 'min_child_samples': 20,
            'subsample': 0.85, 'subsample_freq': 1
        }, 
        'X_num': X_num, 'X_cat': X_cat + X_cat2, 
        'train_data_proc_param': {'org': df_org},
        'validation_fraction': 'oof'
    }, {**config, 'return_train_scores': False}, lgb_adapter, rerun = True
)

In [72]:
np.mean(result['valid_scores']), result['valid_scores']

(0.35795977777777777,
 [0.35725777777777773,
  0.35897555555555555,
  0.35711777777777776,
  0.3574055555555556,
  0.3590422222222222])

In [2]:
models = [
    'xgb1_org', 'xgb2_org', 'xgb3_org', 'xgb4_org', 
    'lgb1_org', 'lgb2_org', 'lgb3_org', 'xgb2_org_cat2', 'lgb2_org_cat2', 'lgb4_org'
]

In [4]:
X_stk = pd.MultiIndex.from_product([models, [str(i) for i in np.arange(cvs[0].shape[1] - 1)]]).to_series().map(lambda x: '_'.join(x)).tolist()
df_cvs = pd.DataFrame(
    np.hstack([sc.read_prd(i)[:, :-1] for i in models]), index = df_train.index, columns = X_stk
)

In [29]:
models_sel = ['xgb4_org', 'xgb3_org', 'xgb2_org_cat2', 'lgb4_org', 'lgb2_org_cat2']

In [23]:
X_stk_sel = pd.MultiIndex.from_product([models_sel, [str(i) for i in np.arange(df_train[target_l].nunique() - 1)]]).to_series().map(lambda x: '_'.join(x)).tolist()
results = sc.cv_result('lr_e_1', df_cvs.assign(**{target_l: df_train[target_l]}), skf, {'X_num': X_stk_sel}, config, lr_adapter)
np.mean(results['valid_scores']), results['valid_scores']

(0.3668126666666666,
 [0.3658588888888888,
  0.3679788888888889,
  0.3661022222222223,
  0.3663888888888889,
  0.36773444444444436])

In [36]:
result = sc.train_cv('lr_e_1', df_cvs.assign(**{target_l: df_train[target_l]}), config)

## Ensemble 1

In [27]:
for i in models_sel:
    sc.train_cv(i, df_train, config, use_gpu = not i.startswith('lgb'), retrain = True)

In [28]:
for i in models_sel:
    sc.cache_result(
        i + '_test',
        lambda : sc.get_predictor_cv(i, config)(df_test).values, rerun = False
    )

/home/sun9sun9/python312/lib/python3.12/site-packages/xgboost/core.py:729: UserWarning: [19:25:39] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/sun9sun9/python312/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [34]:
X_stk_sel = pd.MultiIndex.from_product([models_sel, [str(i) for i in np.arange(df_train[target_l].nunique() - 1)]]).to_series().map(lambda x: '_'.join(x)).tolist()
df_test_stk = pd.DataFrame(
    np.hstack([sc.read_result(i + '_test')[:, :-1] for i in models_sel]), index = df_test.index, columns = X_stk_sel
)
df_test_stk

,xgb4_org_0,xgb4_org_1,xgb4_org_2,xgb4_org_3,xgb4_org_4,xgb4_org_5,xgb3_org_0,xgb3_org_1,xgb3_org_2,xgb3_org_3,...,lgb4_org_2,lgb4_org_3,lgb4_org_4,lgb4_org_5,lgb2_org_cat2_0,lgb2_org_cat2_1,lgb2_org_cat2_2,lgb2_org_cat2_3,lgb2_org_cat2_4,lgb2_org_cat2_5
id,,,,,,,,,,,,,,,,,,,,,
750000,0.202247,0.090522,0.112463,0.162642,0.156720,0.164002,0.230043,0.115597,0.106750,0.161712,...,0.104572,0.180035,0.170043,0.154513,0.155366,0.092870,0.103049,0.167522,0.175629,0.179162
750001,0.150547,0.061383,0.264040,0.159454,0.121274,0.092628,0.151771,0.062426,0.242309,0.145264,...,0.259340,0.155672,0.117295,0.079360,0.126803,0.082898,0.251395,0.176052,0.100918,0.102622
750002,0.109567,0.088855,0.127697,0.218300,0.173145,0.141964,0.118997,0.131817,0.143916,0.188094,...,0.132810,0.217521,0.160367,0.127137,0.137174,0.100119,0.133380,0.180902,0.146951,0.142743
750003,0.122574,0.242916,0.155343,0.121650,0.057499,0.175477,0.155079,0.248141,0.149239,0.105904,...,0.156831,0.110040,0.058183,0.149298,0.109405,0.260004,0.128605,0.104688,0.054252,0.180390
750004,0.143170,0.053672,0.099536,0.243490,0.128469,0.103126,0.116249,0.064298,0.098732,0.282881,...,0.110453,0.246129,0.112838,0.107729,0.170051,0.067705,0.086037,0.264582,0.113058,0.109605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.066473,0.144560,0.157622,0.122388,0.170050,0.084092,0.089482,0.150947,0.130637,0.119963,...,0.166663,0.123729,0.145228,0.083090,0.099238,0.130167,0.127945,0.113067,0.198097,0.088483
999996,0.193987,0.162121,0.140715,0.092751,0.152798,0.123789,0.158163,0.174694,0.164461,0.096813,...,0.141755,0.102750,0.152876,0.108096,0.173097,0.141443,0.133247,0.102338,0.177064,0.133361
999997,0.173832,0.133104,0.113026,0.116661,0.097482,0.202426,0.164050,0.155725,0.131655,0.128731,...,0.121051,0.128320,0.094255,0.164260,0.181366,0.138024,0.113482,0.141493,0.097404,0.170426


In [40]:
df_prd = sc.get_predictor_cv('lr_e_1', config)(df_test_stk)
pd.DataFrame(
    le.inverse_transform(np.argsort(-df_prd.values, axis=1)[:, :3].reshape(-1)).reshape(-1, 3),
    index = df_test.index
).apply(lambda x: x.str.cat(sep=' '), axis=1).rename(target.replace('_', ' ')).to_csv('result/submission2.csv')

In [41]:
#!kaggle competitions submit -c playground-series-s5e6 -f result/submission2.csv -m "Ensemble1"

100%|██████████████████████████████████████| 6.82M/6.82M [00:02<00:00, 2.67MB/s]
Successfully submitted to Predicting Optimal Fertilizers

- 0.36716